# Run Agent
- Implement a workflow to write a daily AI newsletter
- see README.md for details



```mermaid
  flowchart TD
      Start([Start Newsletter Workflow]) --> Step1[1: Gather URLs<br/>RSS + HTML + REST APIs<br/>17+ sources]

      Step1 --> Step2[2: Filter URLs<br/>Dedupe Previously Seen<br/>LLM AI Filter<br/>]

      Step2 --> Step3[3: Download AI Articles<br/>Extract Text]

      Step3 --> Step4[4: LLM Summarization<br/>Bullet-point summaries]

      Step4 --> Step5[5: Cluster by Topic<br/>HDBSCAN + Embeddings]

      Step5 --> Step6[6: Rate Articles<br/>Identify<br/>On-Topic, Quality<br/>Using Elo 1-1 Comparisons]

      Step6 --> Step7[7: Select Sections<br/>Top Articles by Topic<br/>Newsletter Outline]

      Step7 --> Step8[8: Draft Sections<br/>Write Headlines & Content<br/>Run Critic-Optimizer Loop]

      Step8 --> Critique8[Section Critique<br/>Quality Evaluation]
      Critique8 --> Check8{Score >= 8.0?}
      Check8 -->|No & iter < 3| Improve8[Apply Improvements]
      Improve8 --> Critique8
      Check8 -->|Yes or iter = 3| Step9[9: Finalize Newsletter<br/>Generate Title<br/>Assemble Sections]

      Step9 --> Critique9[Full Newsletter<br/>Critic-Optimizer Loop<br/>Evaluate Quality<br/>5 dimension scores]

      Critique9 --> Check9{Score >= 8.0<br/>OR<br/>iter = 3?}

      Check9 -->|No| Optimizer[Optimizer Agent<br/>Apply Recommendations<br/>Improve Newsletter]
      Optimizer --> Critique9

      Check9 -->|Yes| Final([Publication-Ready<br>Newsletter<br/>Quality Score >= 8.0])

      style Step1 fill:#e1f5ff
      style Step2 fill:#e1f5ff
      style Step3 fill:#e1f5ff
      style Step4 fill:#fff9c4
      style Step5 fill:#fff9c4
      style Step6 fill:#fff9c4
      style Step7 fill:#e8f5e9
      style Step8 fill:#e8f5e9
      style Critique8 fill:#ffe0e0
      style Check8 fill:#ffe0e0
      style Improve8 fill:#ffe0e0
      style Step9 fill:#f3e5f5
      style Critique9 fill:#f3e5f5
      style Check9 fill:#f3e5f5
      style Optimizer fill:#f3e5f5
  ```

In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url, AgentState

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            2.6.1
OpenAI Agents SDK  0.4.2
Pydantic           2.12.3


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# turn off open telemetry (langfuse)
os.environ["OTEL_SDK_DISABLED"] = "true"

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger
debuglevel = logging.DEBUG

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig()
    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(debuglevel)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(debuglevel)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(debuglevel)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.debug("Test debug message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

17:59:12 | NewsletterAgent.newsletter_agent | DEBUG | Test debug message
17:59:12 | NewsletterAgent.newsletter_agent | INFO | Test info message
17:59:12 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
17:59:12 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

In [5]:
# look up recent sessions
conn = sqlite3.connect("newsletter_agent.db")
AgentState.list_sessions(conn, n_records=10)


['test_newsletter_20251215174315890777',
 'test_newsletter_20251215083959492064',
 'test_newsletter_20251214193031185704',
 'test_newsletter_20251214151829206863',
 'test_newsletter_20251214154532420691',
 'test_newsletter_20251214104035547716',
 'test_newsletter_20251214103817645475',
 'test_newsletter_20251214101020159548',
 'test_newsletter_20251213094226062031',
 'test_newsletter_20251213090105266727']

In [6]:
z = AgentState.get_latest_by_session(conn, "test_newsletter_20251215083959492064")
z.step_name

'finalize_newsletter'

In [7]:
for y in z.get_all_by_session(conn, "test_newsletter_20251215083959492064"):
    print(y.session_id, y.step_name, y.updated_at)

test_newsletter_20251215083959492064 initialize 2025-12-15 08:39:59.505083
test_newsletter_20251215083959492064 gather_urls 2025-12-15 08:40:52.630036
test_newsletter_20251215083959492064 check_workflow_status 2025-12-15 08:40:53.839388
test_newsletter_20251215083959492064 filter_urls 2025-12-15 08:41:48.191939
test_newsletter_20251215083959492064 download_articles 2025-12-15 08:46:59.981285
test_newsletter_20251215083959492064 extract_summaries 2025-12-15 08:48:26.324954
test_newsletter_20251215083959492064 rate_articles 2025-12-15 08:50:24.954873
test_newsletter_20251215083959492064 cluster_topics 2025-12-15 17:13:36.280925
test_newsletter_20251215083959492064 select_sections 2025-12-15 17:17:09.111831
test_newsletter_20251215083959492064 draft_sections 2025-12-15 17:27:09.605056
test_newsletter_20251215083959492064 finalize_newsletter 2025-12-15 17:40:17.447828


# Run Agent Worfklow

In [8]:
print("🚀 Creating NewsletterAgent...")
do_download=False
# reprocess_since=None
reprocess_since='2025-12-15 08:42:48.191939'
try:
    # set up state
    session_id = 'test_newsletter_20251215083959492064'
    step_name = 'cluster_topics'
    del session_id
except Exception as e:
    print(e)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 reprocess_since=reprocess_since,
                                 verbose=False
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 reprocess_since=reprocess_since,
                                 verbose=False
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

🚀 Creating NewsletterAgent...
session_id is not defined
test_newsletter_20251215175959759192


In [9]:
agent.state.get_status()


{'headlines': {'total': 0},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'gather_urls',
  'workflow_complete': False,
  'progress_percentage': 0.0,
  'completed_steps': 0,
  'total_steps': 9,
  'max_edits': 2,
  'concurrency': 16,
  'do_download': False,
  'reprocess_since': datetime.datetime(2025, 12, 15, 8, 42, 48, 191939)},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [10]:
for step in agent.state.steps:
    print(step)
    print("---")


⚪ Gather URLs [not_started]
   
---
⚪ Filter URLs [not_started]
   
---
⚪ Download Articles [not_started]
   
---
⚪ Extract Summaries [not_started]
   
---
⚪ Rate Articles [not_started]
   
---
⚪ Cluster By Topic [not_started]
   
---
⚪ Select Sections [not_started]
   
---
⚪ Draft Sections [not_started]
   
---
⚪ Finalize Newsletter [not_started]
   
---


In [11]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


18:00:04 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Starting check_workflow_status
18:00:04 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Completed check_workflow_status


⏱️  Total execution time: 9.20s
📊 Final result:
I checked the workflow status.

Summary:
- Progress: 0.0% (0/9 complete)
- Status: 0 complete, 0 started, 0 failed, 9 not started
- Next step: Gather URLs (Step 1)

Step-by-step (all not started):
1. Gather URLs
2. Filter URLs (AI-only)
3. Download Articles
4. Extract Summaries
5. Rate Articles
6. Cluster By Topic
7. Select Sections
8. Draft Sections
9. Finalize Newsletter

What would you like me to do next?
- Run all remaining steps and create the newsletter
- Start the next step (Gather URLs)
- Run a specific step (tell me which)
- Inspect detailed state for debugging


In [12]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


18:00:14 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Starting check_workflow_status
18:00:14 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Completed check_workflow_status
18:00:17 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Starting Step 1: Gather URLs
2025-12-15 18:00:17,105 - fetcher_5504042704 - INFO - [fetcher_init] Loading sources from sources.yaml
2025-12-15 18:00:17,122 - fetcher_5504042704 - INFO - [fetcher_init] Loaded 17 sources: 6 RSS, 10 HTML, 1 API
2025-12-15 18:00:17,123 - fetcher_5504042704 - DEBUG - [fetcher_sources] Source 'Ars Technica': type=RSS, url=https://arstechnica.com/ai/
2025-12-15 18:00:17,123 - fetcher_5504042704 - DEBUG - [fetcher_sources] Source 'Bloomberg': type=RSS, url=https://www.bloomberg.com/ai
2025-12-15 18:00:17,123 - fetcher_5504042704 - DEBUG - [fetcher_sources] Source 'Business Insider': type=html, url=https://www.businessinsider.com/tech
2025-12-15 18:00:17,124 - fetcher_5504042704 - DEB

▶ Starting Step 1: gather_urls


2025-12-15 18:00:17,331 - fetcher_5504042704 - INFO - [fetch_html] Parsed HTML file: download/sources/The_Verge.html
2025-12-15 18:00:17,331 - fetcher_5504042704 - INFO - [fetch_html] HTML fetch successful for The Verge: 28 articles
2025-12-15 18:00:17,331 - fetcher_5504042704 - INFO - [fetch_html] Using existing HTML file from VentureBeat: https://venturebeat.com/category/ai
2025-12-15 18:00:17,331 - fetcher_5504042704 - INFO - [fetch_html] Parsing HTML file: download/sources/VentureBeat.html
2025-12-15 18:00:17,339 - fetcher_5504042704 - INFO - [fetch_html] Parsed HTML file: download/sources/VentureBeat.html
2025-12-15 18:00:17,339 - fetcher_5504042704 - INFO - [fetch_html] HTML fetch successful for VentureBeat: 19 articles
2025-12-15 18:00:17,339 - fetcher_5504042704 - INFO - [fetch_html] Using existing HTML file from WSJ: https://www.wsj.com/tech/ai
2025-12-15 18:00:17,339 - fetcher_5504042704 - INFO - [fetch_html] Parsing HTML file: download/sources/WSJ.html
2025-12-15 18:00:17,36

,source,url
0,Ars Technica,20
1,Bloomberg,25
2,Business Insider,16
3,FT,46
4,Feedly AI,92
5,Hacker News,29
6,HackerNoon,50
7,New York Times,28
8,NewsAPI,41
9,Reddit,52


,source,title,url,published,rss_summary,id
0,Ars Technica,Merriam-Webster’s word of the year delivers a ...,https://arstechnica.com/ai/2025/12/merriam-web...,"Mon, 15 Dec 2025 22:41:43 +0000",Dictionary codifies the term that took hold in...,0
1,Ars Technica,Murder-suicide case shows OpenAI selectively h...,https://arstechnica.com/tech-policy/2025/12/op...,"Mon, 15 Dec 2025 20:10:25 +0000",OpenAI accused of hiding full ChatGPT logs in ...,1
2,Ars Technica,OpenAI built an AI coding agent and uses it to...,https://arstechnica.com/ai/2025/12/how-openai-...,"Fri, 12 Dec 2025 22:16:42 +0000","""The vast majority of Codex is built by Codex,...",2
3,Ars Technica,Google Translate expands live translation to a...,https://arstechnica.com/google/2025/12/google-...,"Fri, 12 Dec 2025 20:44:35 +0000",Expanded live translation will come to iOS in ...,3
4,Ars Technica,Scientists built an AI co-pilot for prosthetic...,https://arstechnica.com/ai/2025/12/scientists-...,"Fri, 12 Dec 2025 19:14:31 +0000","Managing each finger separately can, with the ...",4
...,...,...,...,...,...,...
702,NewsAPI,AI Favors Ecommerce Giants — for Now,https://www.practicalecommerce.com/ai-favors-e...,2025-12-14T18:40:59Z,NaN,702
703,NewsAPI,Key Account Marketing Software Market: 2026 Vi...,https://www.linkedin.com/pulse/key-account-mar...,2025-12-14T22:45:26Z,NaN,703
704,NewsAPI,"Music Marketing Market: Key Drivers, Trends & ...",https://www.linkedin.com/pulse/music-marketing...,2025-12-14T20:09:00Z,NaN,704
705,NewsAPI,Singapore firms: Steady pay rises in 2026,https://www.straitstimes.com/business/spore-fi...,2025-12-14T21:05:32Z,NaN,705


18:00:17 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Completed Step 1: Gathered 838 articles
18:00:19 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Starting check_workflow_status
18:00:19 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Completed check_workflow_status


⏱️  Total execution time: 9.61s
📊 Final result:
Step 1 (Gather URLs) has been run.

Results:
- Step 1 completed successfully.
- Collected 838 article items from 17 sources; 707 articles stored in persistent state.
- Workflow progress: 11.1% (1/9 complete).
- Next step: Step 2 — Filter URLs (AI-only).

Would you like me to:
- Continue to Step 2 now (filter AI-related URLs)?
- Inspect the stored URLs/articles before filtering?
- Run a different step?


In [13]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
10,Techmeme,166
4,Feedly AI,92
9,Reddit,52
12,The Register,50
6,HackerNoon,50
3,FT,46
8,NewsAPI,41
5,Hacker News,29
7,New York Times,28
1,Bloomberg,25


In [14]:
state.print_workflow_status()



📊 WORKFLOW STATUS
📈 Progress: 11.1% (1/9 complete)
➡️  Next step: Step 1: Filter URLs

📋 Step Details:
  ✅ Step 0: Gather URLs            | complete
      📝 Status: ✅ Step 1 gather_urls completed successfully! Gathered 838 articles from 17 sources.

📊 Articles stored in persistent state: 707
  ⭕ Step 1: Filter URLs            | not_started
  ⭕ Step 2: Download Articles      | not_started
  ⭕ Step 3: Extract Summaries      | not_started
  ⭕ Step 4: Rate Articles          | not_started
  ⭕ Step 5: Cluster By Topic       | not_started
  ⭕ Step 6: Select Sections        | not_started
  ⭕ Step 7: Draft Sections         | not_started
  ⭕ Step 8: Finalize Newsletter    | not_started



In [15]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)
# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


18:00:21 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Starting Step 2: Filter URLs
18:00:21 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | 🔍 Filtering 707 headlines...
18:00:21 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | 🔄 Checking for duplicates seen before 2025-12-15T08:42:48.191939
18:00:21 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | 🔍 Filtering 707 articles for dupes.
18:00:22 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | new - keeping https://arstechnica.com/ai/2025/12/merriam-webster-crowns-slop-word-of-the-year-as-ai-content-floods-internet/
18:00:22 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | found after cutoff - keeping https://arstechnica.com/tech-policy/2025/12/openai-refuses-to-say-where-chatgpt-logs-go-when-users-die/
18:00:22 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | found before cutoff - ignoring as duplicate https://arstechnica.com/ai/2

▶ Starting Step 2: filter_urls


18:00:22 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | found before cutoff - ignoring as duplicate https://www.ft.com/content/cf9e481e-a316-4b97-a454-044b2cae6716
18:00:22 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | found before cutoff - ignoring as duplicate https://www.ft.com/content/ff72f64b-fc58-4f11-9b75-e30f99031936
18:00:22 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | found after cutoff - keeping https://www.ft.com/content/0e16b6ea-bf46-4ea6-820e-244b7a5bae1d
18:00:22 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | found before cutoff - ignoring as duplicate https://www.ft.com/content/cd171921-a0f5-49d9-a383-41a37e34dbb4
18:00:22 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | found before cutoff - ignoring as duplicate https://www.ft.com/content/294c24e6-0647-4ebb-ab08-278061e8fe19
18:00:22 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | found after cutoff - keeping https://

⏱️  Total execution time: 120.08s
📊 Final result:
✅ Step 2 filter_urls completed successfully! Removed 484 duplicate URLs, classified 223 new articles, found 103 AI-related.


In [16]:
headline_df = pd.DataFrame(state.headline_data)
headline_df


,source,title,url,published,rss_summary,id,isAI
0,Ars Technica,Merriam-Webster’s word of the year delivers a ...,https://arstechnica.com/ai/2025/12/merriam-web...,"Mon, 15 Dec 2025 22:41:43 +0000",Dictionary codifies the term that took hold in...,0,True
1,Ars Technica,Murder-suicide case shows OpenAI selectively h...,https://arstechnica.com/tech-policy/2025/12/op...,"Mon, 15 Dec 2025 20:10:25 +0000",OpenAI accused of hiding full ChatGPT logs in ...,1,True
2,Bloomberg,The Big TakeWhat 2025’s Economic Warning Signs...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,20,True
3,Bloomberg,Musk’s Fortune Soars Above $600 Billion on New...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,21,False
4,Bloomberg,OpenAI Deal to License Disney Characters Is En...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,22,True
...,...,...,...,...,...,...,...
218,NewsAPI,AI Favors Ecommerce Giants — for Now,https://www.practicalecommerce.com/ai-favors-e...,2025-12-14T18:40:59Z,NaN,702,True
219,NewsAPI,Key Account Marketing Software Market: 2026 Vi...,https://www.linkedin.com/pulse/key-account-mar...,2025-12-14T22:45:26Z,NaN,703,False
220,NewsAPI,"Music Marketing Market: Key Drivers, Trends & ...",https://www.linkedin.com/pulse/music-marketing...,2025-12-14T20:09:00Z,NaN,704,False
221,NewsAPI,Singapore firms: Steady pay rises in 2026,https://www.straitstimes.com/business/spore-fi...,2025-12-14T21:05:32Z,NaN,705,False


In [17]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


18:02:28 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Starting Step 3: Download Articles
18:02:28 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Starting concurrent scraping of 103 AI-related articles
18:02:28 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Launching browser for 103 URLs with 16 concurrent workers


▶ Starting Step 3: download_articles


18:02:29 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Worker 0 fetching 1 of 103 https://venturebeat.com/ai/nvidia-debuts-nemotron-3-with-hybrid-moe-and-mamba-transformer-to-drive
18:02:29 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | scrape_url(https://venturebeat.com/ai/nvidia-debuts-nemotron-3-with-hybrid-moe-and-mamba-transformer-to-drive)
18:02:29 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | scraping https://venturebeat.com/ai/nvidia-debuts-nemotron-3-with-hybrid-moe-and-mamba-transformer-to-drive to download/html
18:02:29 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Downloading https://venturebeat.com/ai/nvidia-debuts-nemotron-3-with-hybrid-moe-and-mamba-transformer-to-drive
18:02:29 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Worker 1 fetching 2 of 103 https://www.businessinsider.com/most-common-ai-use-workplace-gallup-poll-2025-12
18:02:29 | NewsletterAgent.test_newsletter_20251215175

Starting with 102 rows...
Processing 102 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2

Creating indexed similarity matrix...
Finding pairs with similarity > 0.925...
Filtering dataframe...
  Pair: 48 vs 70
    48: Feedly AI - Sorry six-figure earners: Elon Musk say that money will 'disappear' in the future as AI makes work (and salaries) irrelevant
    48: https://fortune.com/2025/12/15/billionaire-elon-musk-say-that-money-will-disappear-in-the-future-as-ai-makes-work-and-salaries-irrelevant-sorry-six-figure-earners/
    70: Feedly AI - Sorry, six-figure earners: Elon Musk says that money will ‘disappear’ in the future as AI makes work (and salaries) irrelevant
    70: https://finance.yahoo.com/news/sorry-six-figure-earners-elon-135903085.html
  Pair: 23 vs 87
    23: Feedly AI - IBM unleashes CUGA, an open-source AI agent that actually completes more than half its tasks
    23: https://www.theregister.com/2025/12/15/ibm_cuga_agent/
    87: The Register - IBM unleashes CUGA, an open-source AI agent that actually completes more than half its tasks
    87: https://go.there

,title,status,final_url
38,Mark Cuban Sends Strong Message to Creatives About AI,403,https://www.inc.com/ava-levinson/mark-cuban-sends-strong-message-to-creatives-about-ai/91278961
6,YouTube atrae a más niños de 1 a 3 años y creadores buscan lucrarse con contenido generado por IA,skipped,https://www.bloomberg.com/news/articles/2025-12-15/youtube-y-la-ia-auge-del-contenido-para-bebes-preocupa-a-expertos-y-padres
2,"The Big TakeWhat 2025’s Economic Warning Signs Mean for 2026Today on the Big Take: All year, the jobs market, consumer sentiment, AI and inflation flashed economic warning signs — what does this mean for 2026?",skipped,https://www.bloomberg.com/news/articles/2025-12-15/what-2025-s-jobs-inflation-and-ai-risks-mean-for-the-economy-in-2026
3,OpenAI Deal to License Disney Characters Is Entirely in Stock,skipped,https://www.bloomberg.com/news/articles/2025-12-15/openai-deal-to-license-disney-characters-is-entirely-in-stock
4,"Oracle a ‘Show Me Story’ on Big AI Debt Bet, JPMorgan Says",skipped,https://www.bloomberg.com/news/articles/2025-12-15/oracle-a-show-me-story-after-big-ai-debt-bet-jpmorgan-says
5,LSEG Lags Own Index With Investors Impatient for AI Breakthrough,skipped,https://www.bloomberg.com/news/articles/2025-12-15/lseg-lags-own-index-with-investors-impatient-for-ai-breakthrough
68,Most Enterprise AI Users Save Just an Hour of Work Per Day,403,https://www.extremetech.com/computing/most-enterprise-ai-users-save-just-an-hour-of-work-per-day
8,AI Drug-Discovery Firm Chai Raises $130 Million Series B,skipped,https://www.bloomberg.com/news/videos/2025-12-15/ai-drug-discovery-firm-chai-raises-130-million-series-b-video
92,AI Is About to Empty Madison Avenue,skipped,https://www.wsj.com/opinion/ai-is-about-to-empty-madison-avenue-58ab2ea2
7,"Wall Street Bets on If, When AI Bubble Will Pop | Bloomberg Tech 12/15/2025",skipped,https://www.bloomberg.com/news/videos/2025-12-15/bloomberg-tech-12-15-2025-video


18:07:00 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Completed Step 3: Downloaded 86 articles


⏱️  Total execution time: 271.83s
📊 Final result:
✅ Step 3 download_articles completed successfully! Downloaded 86 AI-related articles with 86% success rate.
📊 Average article length: 4653 characters
🔗 Content stored in persistent state.


In [18]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["html_path"]==""])


,id,source,title,status,final_url
83,145,Feedly AI,Mark Cuban Sends Strong Message to Creatives About AI,403,https://www.inc.com/ava-levinson/mark-cuban-sends-strong-message-to-creatives-about-ai/91278961
84,32,Bloomberg,YouTube atrae a más niños de 1 a 3 años y creadores buscan lucrarse con contenido generado por IA,skipped,https://www.bloomberg.com/news/articles/2025-12-15/youtube-y-la-ia-auge-del-contenido-para-bebes-preocupa-a-expertos-y-padres
85,20,Bloomberg,"The Big TakeWhat 2025’s Economic Warning Signs Mean for 2026Today on the Big Take: All year, the jobs market, consumer sentiment, AI and inflation flashed economic warning signs — what does this mean for 2026?",skipped,https://www.bloomberg.com/news/articles/2025-12-15/what-2025-s-jobs-inflation-and-ai-risks-mean-for-the-economy-in-2026
86,22,Bloomberg,OpenAI Deal to License Disney Characters Is Entirely in Stock,skipped,https://www.bloomberg.com/news/articles/2025-12-15/openai-deal-to-license-disney-characters-is-entirely-in-stock
87,24,Bloomberg,"Oracle a ‘Show Me Story’ on Big AI Debt Bet, JPMorgan Says",skipped,https://www.bloomberg.com/news/articles/2025-12-15/oracle-a-show-me-story-after-big-ai-debt-bet-jpmorgan-says
88,26,Bloomberg,LSEG Lags Own Index With Investors Impatient for AI Breakthrough,skipped,https://www.bloomberg.com/news/articles/2025-12-15/lseg-lags-own-index-with-investors-impatient-for-ai-breakthrough
89,190,Feedly AI,Most Enterprise AI Users Save Just an Hour of Work Per Day,403,https://www.extremetech.com/computing/most-enterprise-ai-users-save-just-an-hour-of-work-per-day
90,35,Bloomberg,AI Drug-Discovery Firm Chai Raises $130 Million Series B,skipped,https://www.bloomberg.com/news/videos/2025-12-15/ai-drug-discovery-firm-chai-raises-130-million-series-b-video
91,617,WSJ,AI Is About to Empty Madison Avenue,skipped,https://www.wsj.com/opinion/ai-is-about-to-empty-madison-avenue-58ab2ea2
92,34,Bloomberg,"Wall Street Bets on If, When AI Bubble Will Pop | Bloomberg Tech 12/15/2025",skipped,https://www.bloomberg.com/news/videos/2025-12-15/bloomberg-tech-12-15-2025-video


In [19]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

18:07:00 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Processing 100 AI articles for summarization
18:07:00 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
18:07:00 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Parsed prompt 'newsagent/extract_summaries': model=gpt-5-mini, reasoning_effort=medium, system_len=1273, user_len=43
18:07:00 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Using model 'gpt-5-mini' for summarization
18:07:00 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Starting summarization for 100 articles


▶ Starting Step 4: extract_summaries


18:07:27 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Extracting metadata from HTML files for 100 articles
18:07:30 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Successfully retrieved prompt 'newsagent/item_distiller' from Langfuse
18:07:30 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Parsed prompt 'newsagent/item_distiller': model=gpt-5-mini, reasoning_effort=medium, system_len=1636, user_len=191


⏱️  Total execution time: 53.81s
📊 Final result:
✅ Step 4 extract_summaries completed successfully! Generated AI-powered summaries for 100/100 articles.
💾 Summaries stored in headline DataFrame.


In [20]:
# if we get a refusal, examine why and delete bad rows
bad_stuff =["Anthropic's Jack Cla",
           ]
print([len(s) for s in bad_stuff])
headline_df = state.headline_df
headline_df.loc[(headline_df["title"].str[:20].isin(bad_stuff))]



[20]


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation,summary,description,tags,short_summary


In [21]:
# remove bad stuff
state.headline_df_to_dict(headline_df.loc[~headline_df["title"].str[:20].isin(bad_stuff)])


In [22]:
# User prompt to run workflow
# user_prompt = "Run step 5, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


18:08:28 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Rating 100 AI articles using fn_rate_articles
18:08:28 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Calculating article ratings for 100 articles
18:08:28 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Rating recency
18:08:28 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Rating spam probability
18:08:28 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
18:08:29 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, reasoning_effort=medium, system_len=1849, user_len=246


▶ Starting Step 5: rate_articles


18:08:32 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | low quality articles: {0.0: 75, 1.0: 15, 0.0001233946226544279: 1, 9.357622968840175e-14: 1, 0.999512127527129: 1, 2.6810038677818034e-14: 1, 1.5428112031918877e-13: 1, 5.3157852544244216e-08: 1, 4.139937718785167e-08: 1, 0.9840936132099872: 1, 0.6791786762514135: 1}
18:08:32 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
18:08:32 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, reasoning_effort=medium, system_len=1790, user_len=240
18:08:35 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | on topic articles: {1.0: 74, 0.9626729941518225: 2, 0.9988304590053244: 2, 0.002472623024627213: 2, 0.9933071269064123: 2, 0.047425874568977164: 1, 7.484619074298232e-05: 1, 1.0129987330277146e-05: 1, 0.0031726831078774057: 1, 0.006692851264215723: 1, 0.09534

,id,site_name,title,bradley_terry,bt_z
56,56,Bored Panda,“Scarier Than Terminator”: ChatGPT Accused Of Turning Son Against His Mother Before Horrifying Final Act,24.092253,1.489021
13,13,TechCrunch,Nvidia bulks up open source offerings with an acquisition and new open AI models,24.092253,1.489021
52,52,VentureBeat,"Nvidia launches Nemotron 3, a family of AI models using a hybrid mixture-of-experts architecture and the Mamba-Transformer design, in 30B, 100B, and ~500B sizes",24.092253,1.489021
62,62,247wallst,Nvidia’s Biggest AI Bet Is Crashing Its ‘Secret Portfolio.’ Can It Recover?,24.092253,1.489021
42,42,Fortune,Sorry six-figure earners: Elon Musk say that money will 'disappear' in the future as AI makes work (and salaries) irrelevant,24.092253,1.489021
...,...,...,...,...,...
58,58,TS2 Tech,"S&P 500 Today (Dec. 15, 2025): Index Holds Near 6,800s as Payrolls, Inflation and AI Anxiety Set the Tone",-24.092253,-1.489021
63,63,Futurity,AI can learn cultural values the way kids do,-24.092253,-1.489021
38,38,Mashable,Forget the em dash — Redditors share 6 giveaways that something was written by AI,-24.092253,-1.489021
46,46,Bloomberg,LSEG Lags Own Index With Investors Impatient for AI Breakthrough,-24.092253,-1.489021


18:08:46 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Top 10 ids: [56, 13, 52, 62, 42, 66, 36, 30, 21, 82]
18:08:46 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | After round 1/20: 
18:08:46 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Number of ranking changes: 98
18:08:46 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Sum of absolute ranking changes: 3311.0 (avg rank chg 33.44)
18:08:46 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | ---------------------------------------------------
18:08:46 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Running round 2 of max 20
18:08:46 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | ---------------------------------------------------
18:08:46 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Generated 17 battle batches
18:08:46 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Total stories in batch

...

,id,site_name,title,bradley_terry,bt_z
56,56,Bored Panda,“Scarier Than Terminator”: ChatGPT Accused Of Turning Son Against His Mother Before Horrifying Final Act,60.361636,4.785263
13,13,TechCrunch,Nvidia bulks up open source offerings with an acquisition and new open AI models,51.339526,4.070021
57,57,NPR,President Trump moves to block state AI laws as New York pushes tougher rules,42.932783,3.403564
52,52,VentureBeat,"Nvidia launches Nemotron 3, a family of AI models using a hybrid mixture-of-experts architecture and the Mamba-Transformer design, in 30B, 100B, and ~500B sizes",34.851187,2.762882
0,0,The New York Times,How Tech’s Biggest Companies Are Offloading the Risks of the A.I. Boom,26.972771,2.138309
...,...,...,...,...,...
51,51,The Wall Street Journal,The Eerie Parallels Between AI Mania and the Dot-Com Bubble,-12.798207,-1.014598
96,96,ExtremeTech,Most Enterprise AI Users Save Just an Hour of Work Per Day,-13.043858,-1.034072
90,90,Fast Company,New survey reveals this surprising fact about companies that successfully use AI,-13.532436,-1.072805
98,98,MarketWatch,These space stocks could benefit from Elon Musk’s latest AI obsession,-14.391877,-1.140939


18:09:51 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Top 10 ids: [56, 13, 57, 52, 0, 59, 21, 66, 65, 62]
18:09:51 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | After round 11/20: 
18:09:51 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Number of ranking changes: 33
18:09:51 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Sum of absolute ranking changes: 948.0 (avg rank chg 9.58)
18:09:51 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | last_two: 8.94, prev_two: 13.00
18:09:51 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | ---------------------------------------------------
18:09:51 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Running round 12 of max 20
18:09:51 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | ---------------------------------------------------
18:09:51 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Generated 21 battle

,id,site_name,title,bradley_terry,bt_z
56,56,Bored Panda,“Scarier Than Terminator”: ChatGPT Accused Of Turning Son Against His Mother Before Horrifying Final Act,60.444074,4.783659
13,13,TechCrunch,Nvidia bulks up open source offerings with an acquisition and new open AI models,51.423400,4.069745
57,57,NPR,President Trump moves to block state AI laws as New York pushes tougher rules,43.018148,3.404538
52,52,VentureBeat,"Nvidia launches Nemotron 3, a family of AI models using a hybrid mixture-of-experts architecture and the Mamba-Transformer design, in 30B, 100B, and ~500B sizes",34.938179,2.765074
0,0,The New York Times,How Tech’s Biggest Companies Are Offloading the Risks of the A.I. Boom,27.061554,2.141703
...,...,...,...,...,...
51,51,The Wall Street Journal,The Eerie Parallels Between AI Mania and the Dot-Com Bubble,-12.791661,-1.012356
96,96,ExtremeTech,Most Enterprise AI Users Save Just an Hour of Work Per Day,-13.049047,-1.032726
90,90,Fast Company,New survey reveals this surprising fact about companies that successfully use AI,-13.581324,-1.074852
98,98,MarketWatch,These space stocks could benefit from Elon Musk’s latest AI obsession,-14.396683,-1.139381


18:09:57 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Top 10 ids: [56, 13, 57, 52, 0, 59, 21, 66, 65, 62]
18:09:57 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | After round 12/20: 
18:09:57 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Number of ranking changes: 37
18:09:57 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Sum of absolute ranking changes: 1174.0 (avg rank chg 11.86)
18:09:57 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | last_two: 10.72, prev_two: 9.47
18:09:57 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Increase in avg rank change, stopping
18:09:57 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Low rated articles: 1
18:09:57 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | low rated article: These space stocks could benefit from Elon Musk’s latest AI obsession -0.45906869086303237
18:09:59 | NewsletterAgent.test_newsletter_20251215

⏱️  Total execution time: 90.20s
📊 Final result:
✅ Step 5 rate_articles completed successfully! Rated 98 articles with average rating 6.2/10.
⭐ High quality articles (≥7.0): 34
💾 Ratings stored in persistent state.


In [23]:
headline_df=state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df.sort_values("rating", ascending=False)[['site_name', 'title', 'rating', 'short_summary']])


,site_name,title,rating,short_summary
13,TechCrunch,Nvidia bulks up open source offerings with an acquisition and new open AI models,13.852823,"Nvidia acquired SchedMD, lead developer of the open-source Slurm workload manager, and launched the Nemotron 3 family of open AI models (Nano, Super, Ultra), saying Slurm will remain open-source and vendor-neutral."
56,Bored Panda,“Scarier Than Terminator”: ChatGPT Accused Of Turning Son Against His Mother Before Horrifying Final Act,13.243672,"On August 5, 2025, in Greenwich, Connecticut, Suzanne Eberson Adams was found killed and her son Stein-Erik Soelberg died by suicide; Adams's estate sued OpenAI and Microsoft in California, alleging ChatGPT reinforced delusions that led to her death."
0,The New York Times,How Tech’s Biggest Companies Are Offloading the Risks of the A.I. Boom,12.906591,"Microsoft, Meta and Google are structuring deals—Microsoft leasing tens of billions of compute, Meta securing nearly $30 billion financing, Google renting and reselling capacity—to shift multibillion-dollar AI data-center build-and-debt risks onto smaller operators and private lenders."
57,NPR,President Trump moves to block state AI laws as New York pushes tougher rules,11.080993,"President Trump issued an executive order to block state AI regulations and prompt attorneys general to sue, while New York Assemblyman Alex Bores backs the RAISE Act requiring frontier AI developers to publish safety plans, report incidents, and gate deployments."
1,The New York Times,"In A.I. Boom, Venture Capital Firms Are Raising Loads More Money",10.442750,"Lightspeed Venture Partners raised over $9 billion—their largest haul—with $3.3 billion earmarked for follow-ons to AI bets including Anthropic, xAI and Mistral, as major VCs similarly raised multibillion-dollar funds amid concentrated venture capital."
...,...,...,...,...
90,Fast Company,New survey reveals this surprising fact about companies that successfully use AI,1.640187,A new survey reports a surprising finding about companies that successfully use AI.
94,Biztoc,Mark Cuban Says 'This Is The Job-Hunting Advice I Give My Own Kids In The Age Of AI',1.203254,"Mark Cuban told Benzinga on Dec. 14, 2025 that job seekers should favor smaller companies in the age of AI because they offer broader responsibilities, faster on‑the‑job learning and opportunities to build cross‑functional experience and AI tool familiarity."
97,Seeking Alpha,xAI having difficulties selling Grok to enterprises: report (X.AI:Private),0.828876,"xAI is having difficulty selling its Grok chatbot to enterprise customers, according to a report."
96,ExtremeTech,Most Enterprise AI Users Save Just an Hour of Work Per Day,0.707596,Most enterprise AI users save about one hour of work per day by using AI tools.


In [24]:
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"""
{row.rating:.1f} - [{row.title}]({row.url}) - {row.site_name}

{row.short_summary}
    """))
    


13.9 - [Nvidia bulks up open source offerings with an acquisition and new open AI models](https://techcrunch.com/2025/12/15/nvidia-bulks-up-open-source-offerings-with-an-acquisition-and-new-open-ai-models/) - TechCrunch

Nvidia acquired SchedMD, lead developer of the open-source Slurm workload manager, and launched the Nemotron 3 family of open AI models (Nano, Super, Ultra), saying Slurm will remain open-source and vendor-neutral.
    


13.2 - [“Scarier Than Terminator”: ChatGPT Accused Of Turning Son Against His Mother Before Horrifying Final Act](https://www.boredpanda.com/chatgpt-accused-of-being-complicit-in-fatal-attack/) - Bored Panda

On August 5, 2025, in Greenwich, Connecticut, Suzanne Eberson Adams was found killed and her son Stein-Erik Soelberg died by suicide; Adams's estate sued OpenAI and Microsoft in California, alleging ChatGPT reinforced delusions that led to her death.
    


12.9 - [How Tech’s Biggest Companies Are Offloading the Risks of the A.I. Boom](https://www.nytimes.com/2025/12/15/technology/ai-risks-debt.html) - The New York Times

Microsoft, Meta and Google are structuring deals—Microsoft leasing tens of billions of compute, Meta securing nearly $30 billion financing, Google renting and reselling capacity—to shift multibillion-dollar AI data-center build-and-debt risks onto smaller operators and private lenders.
    


11.1 - [President Trump moves to block state AI laws as New York pushes tougher rules](https://www.npr.org/2025/12/14/nx-s1-5644086/president-trump-moves-to-block-state-ai-laws-as-new-york-pushes-tougher-rules) - NPR

President Trump issued an executive order to block state AI regulations and prompt attorneys general to sue, while New York Assemblyman Alex Bores backs the RAISE Act requiring frontier AI developers to publish safety plans, report incidents, and gate deployments.
    


10.4 - [In A.I. Boom, Venture Capital Firms Are Raising Loads More Money](https://www.nytimes.com/2025/12/15/technology/ai-venture-capital-big-funds.html) - The New York Times

Lightspeed Venture Partners raised over $9 billion—their largest haul—with $3.3 billion earmarked for follow-ons to AI bets including Anthropic, xAI and Mistral, as major VCs similarly raised multibillion-dollar funds amid concentrated venture capital.
    

...


1.6 - [New survey reveals this surprising fact about companies that successfully use AI](https://www.fastcompany.com/91460733/ai-jobs-layoffs-productivity-new-survey-reveals-this-surprising-fact-about-companies-that-successfully-use-artificial-intelligence) - Fast Company

A new survey reports a surprising finding about companies that successfully use AI.
    


1.2 - [Mark Cuban Says 'This Is The Job-Hunting Advice I Give My Own Kids In The Age Of AI'](https://biztoc.com/x/dad49cec83da3a72) - Biztoc

Mark Cuban told Benzinga on Dec. 14, 2025 that job seekers should favor smaller companies in the age of AI because they offer broader responsibilities, faster on‑the‑job learning and opportunities to build cross‑functional experience and AI tool familiarity.
    


0.8 - [xAI having difficulties selling Grok to enterprises: report (X.AI:Private)](https://seekingalpha.com/news/4531596-xai-having-difficulties-selling-grok-to-enterprises-report) - Seeking Alpha

xAI is having difficulty selling its Grok chatbot to enterprise customers, according to a report.
    


0.7 - [Most Enterprise AI Users Save Just an Hour of Work Per Day](https://www.extremetech.com/computing/most-enterprise-ai-users-save-just-an-hour-of-work-per-day) - ExtremeTech

Most enterprise AI users save about one hour of work per day by using AI tools.
    


0.6 - [Mark Cuban Sends Strong Message to Creatives About AI](https://www.inc.com/ava-levinson/mark-cuban-sends-strong-message-to-creatives-about-ai/91278961) - Inc.

Mark Cuban sent a public message to creatives about artificial intelligence.
    

In [25]:
# User prompt to run workflow 
# user_prompt = "Run step 6, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


18:11:18 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Starting topic extraction for clustering
18:11:18 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Successfully retrieved prompt 'newsagent/extract_topics' from Langfuse
18:11:18 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Parsed prompt 'newsagent/extract_topics': model=gpt-5-nano, reasoning_effort=medium, system_len=1100, user_len=80
18:11:18 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Using model 'gpt-5-nano' for topic extraction
18:11:18 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Processing 98 articles for topic extraction


▶ Starting Step 6: cluster_topics


18:12:11 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Successfully extracted 338 total topics across articles
18:12:11 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Starting canonical topic classification for 126 topics
18:12:11 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
18:12:11 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Parsed prompt 'newsagent/canonical_topic': model=gpt-5-nano, reasoning_effort=medium, system_len=497, user_len=216
18:12:11 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
18:12:11 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Parsed prompt 'newsagent/canonical_topic': model=gpt-5-nano, reasoning_effort=medium, system_len=497, user_len=216
18:12:11 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Succ

=== HDBSCAN Parameters ===
min_cluster_size:   5
min_samples:        5
=== Clustering Quality Metrics ===
Number of clusters: 3
Noise points: 16 (16.3%)
Average cluster size: 27.3 ± 18.4
Cluster size range: 6 - 51
=== Quality Scores ===
Silhouette Score: 0.347 (higher is better)
Calinski-Harabasz Score: 62.3 (higher is better)
Davies-Bouldin Score: 0.667 (lower is better)
HDBSCAN Validity Index: 0.325
Composite Score: 0.336 (higher is better)

=== HDBSCAN Parameters ===
min_cluster_size:   8
min_samples:        5
=== Clustering Quality Metrics ===
Number of clusters: 2
Noise points: 14 (14.3%)
Average cluster size: 42.0 ± 9.0
Cluster size range: 33 - 51
=== Quality Scores ===
Silhouette Score: 0.460 (higher is better)
Calinski-Harabasz Score: 99.4 (higher is better)
Davies-Bouldin Score: 0.824 (lower is better)
HDBSCAN Validity Index: 0.269
Composite Score: 0.365 (higher is better)

=== HDBSCAN Parameters ===
min_cluster_size:   3
min_samples:        1
=== Clustering Quality Metrics ==

/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
[I 2025-12-15 18:14:28,992] Trial 8 finished with value: -0.3374135063826924 and parameters: {'min_cluster_size': 4, 'min_samples': 3}. Best is trial 3 with value: -0.3775142686949824.
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/en

...

=== Clustering Quality Metrics ===
Number of clusters: 2
Noise points: 31 (31.6%)
Average cluster size: 33.5 ± 9.5
Cluster size range: 24 - 43
=== Quality Scores ===
Silhouette Score: 0.538 (higher is better)
Calinski-Harabasz Score: 111.0 (higher is better)
Davies-Bouldin Score: 0.675 (lower is better)
HDBSCAN Validity Index: 0.310
Composite Score: 0.424 (higher is better)

=== HDBSCAN Parameters ===
min_cluster_size:   9
min_samples:        9
=== Clustering Quality Metrics ===
Number of clusters: 2
Noise points: 31 (31.6%)
Average cluster size: 33.5 ± 9.5
Cluster size range: 24 - 43
=== Quality Scores ===
Silhouette Score: 0.538 (higher is better)
Calinski-Harabasz Score: 111.0 (higher is better)
Davies-Bouldin Score: 0.675 (lower is better)
HDBSCAN Validity Index: 0.310
Composite Score: 0.424 (higher is better)

=== HDBSCAN Parameters ===
min_cluster_size:   10
min_samples:        9
=== Clustering Quality Metrics ===
Number of clusters: 2
Noise points: 31 (31.6%)
Average cluster siz

/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
[I 2025-12-15 18:14:29,888] Trial 40 finished with value: -0.4170248554455068 and parameters: {'min_cluster_size': 10, 'min_samples': 10}. Best is trial 15 with value: -0.4242875263303564.
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda

=== Clustering Quality Metrics ===
Number of clusters: 2
Noise points: 37 (37.8%)
Average cluster size: 30.5 ± 8.5
Cluster size range: 22 - 39
=== Quality Scores ===
Silhouette Score: 0.551 (higher is better)
Calinski-Harabasz Score: 107.9 (higher is better)
Davies-Bouldin Score: 0.650 (lower is better)
HDBSCAN Validity Index: 0.283
Composite Score: 0.417 (higher is better)

=== HDBSCAN Parameters ===
min_cluster_size:   9
min_samples:        9
=== Clustering Quality Metrics ===
Number of clusters: 2
Noise points: 31 (31.6%)
Average cluster size: 33.5 ± 9.5
Cluster size range: 24 - 43
=== Quality Scores ===
Silhouette Score: 0.538 (higher is better)
Calinski-Harabasz Score: 111.0 (higher is better)
Davies-Bouldin Score: 0.675 (lower is better)
HDBSCAN Validity Index: 0.310
Composite Score: 0.424 (higher is better)

=== HDBSCAN Parameters ===
min_cluster_size:   9
min_samples:        8
=== Clustering Quality Metrics ===
Number of clusters: 2
Noise points: 26 (26.5%)
Average cluster size

18:14:38 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | 1: AI's Impact on Society and Business
18:14:38 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | GNOME bans AI-generated extensions (GNOME, Open Source, Linux, Extensions, Gen AI, Code Quality, Governance)
Merriam-Webster’s word of the year delivers a dismissive verdict on junk AI content (Gen AI, AI Content, Content Moderation, Society And Culture, Books And Publishing, Word Choice, Monetization)
Salesforce willing to lose money on AI agent licenses when customers are locked in (Licensing, Deals, Monetization, AI Agents, Enterprise AI, Vendor Lock-in, Adoption and Spending)
More workers are using AI, but don't know if their employers are, too - why that's a problem (Workplace AI, Artificial Intelligence, Adoption and Spending, Jobs And Careers, Governance, Employee Training, Industry Variations)
IBM unleashes CUGA, an open-source AI agent that actually completes more than half its tasks (AI Agents,

⏱️  Total execution time: 204.75s
📊 Final result:
✅ Step 6 cluster_topics completed successfully! Organized 98 articles into topic clusters.


In [26]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("select_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


18:16:38 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Category proposal
18:16:38 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Successfully retrieved prompt 'newsagent/cat_proposal' from Langfuse
18:16:38 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Parsed prompt 'newsagent/cat_proposal': model=gpt-5-mini, reasoning_effort=medium, system_len=638, user_len=1179


▶ Starting Step 7: select_sections


18:17:26 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Cleaning up initial categories: ['OpenAI Withholding Chat Logs', 'AI Infrastructure Boom and Markets', 'Notion $11B Tender Offer', 'Klarna Agentic Product Protocol', 'GNOME Bans AI Extensions', 'Motif Lessons For Enterprise LLMs', 'CDS Hedging Against AI Bubble', 'AI Data Center Financing', 'Rise Of AI Slop', 'IBM Releases CUGA Agent', 'Rivian Large Driving Model Ambition', 'CrowdStrike Falcon AIDR Launch', 'FSOC Flags Cyber-AI Stability Risk', 'Workplace AI Communication Gap', 'Trump Order Blocks State AI', 'Tesla Robotaxis Testing Unsupervised', 'Dell $25B AI Systems Guidance', 'Creative Commons Backs Pay-To-Crawl', 'Other', 'Google Search AI Upload Button', 'VC Megafunds For AI', 'Nearline HDD Price Spike', "AI's Impact on Society and Business", 'Disney One-Year OpenAI Exclusivity', 'Nvidia Open Models & Slurm', 'Salesforce Flat AI Agent Deals']
18:17:26 | NewsletterAgent.test_newsletter_20251215175959759192 | 

⏱️  Total execution time: 131.73s
📊 Final result:
Categories and article counts:
cat
AI Data Center Financing                 6
AI Impact On Business                   33
AI Impact On Society                    11
AI Infrastructure Boom                  16
Disney One-Year OpenAI Exclusivity       2
Motif Enterprise LLM Lessons             2
OpenAI Withholding Chat Logs             3
Other                                    3
Rise Of AI Slop                          4
Tesla Robotaxis Unsupervised Testing     3
Trump Order Blocks State AI              2
VC Megafunds For AI                      4


In [27]:
# User prompt to run workflow
# user_prompt = "Run step 8, draft sections"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("draft_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


18:18:59 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Step 8a: Selecting stories from 89 total
18:18:59 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | 20 must-include stories with rating > 8
18:18:59 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | 69 stories with rating <= 8


▶ Starting Step 8: draft_sections


18:19:00 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Selected 69 diverse Tier 2 stories via MMR
18:19:00 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Total selected stories: 89 (target: 100)
18:19:00 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Drafting sections for 12 categories (including Other) from selected stories
18:19:00 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Successfully retrieved prompt 'newsagent/write_section' from Langfuse
18:19:00 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Parsed prompt 'newsagent/write_section': model=gpt-5-mini, reasoning_effort=high, system_len=1854, user_len=22
18:23:33 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Drafted 12 sections
18:23:33 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Reassigned 9 pruned stories to Other section
18:23:33 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Created ne

⏱️  Total execution time: 535.16s
📊 Final result:
✅ Step 8 draft_sections completed successfully! Drafted 58 stories across 11 sections


In [28]:
newsletter_section_df = state.newsletter_section_df

newsletter_section_df

,cat,section_title,id,headline,rating,prune,links,sort_order
0,AI Data Center Financing,AI Data-Center Debt Squeeze,0,Big Tech shifts AI data-center debt onto small...,12.906591,False,[The New York Times](https://www.nytimes.com/2...,3
1,AI Data Center Financing,AI Data-Center Debt Squeeze,19,Tech CDS buying surges 90% since September as ...,8.119794,False,[Gizmodo](https://gizmodo.com/investors-are-us...,3
2,AI Data Center Financing,AI Data-Center Debt Squeeze,29,JPMorgan calls Oracle a 'show-me' stock after ...,5.720533,False,[Bloomberg](https://www.bloomberg.com/news/art...,3
3,AI Data Center Financing,AI Data-Center Debt Squeeze,62,86% of Nvidia's $3.84B AI portfolio tied to Co...,6.912561,False,[247wallst](https://247wallst.com/investing/20...,3
4,AI Data Center Financing,AI Data-Center Debt Squeeze,43,Data-center energy vendor cites strong AI orde...,6.101751,False,[CNBC](https://www.cnbc.com/2025/12/15/were-bo...,3
5,AI Data Center Financing,AI Data-Center Debt Squeeze,48,"Energy Transfer advances Lake Charles LNG, AI-...",5.962493,False,[Yahoo](https://finance.yahoo.com/news/did-ene...,3
6,VC Megafunds For AI,AI Funding Frenzy,1,"Lightspeed raises $9B, earmarks $3.3B to back ...",10.442750,False,[The New York Times](https://www.nytimes.com/2...,4
7,VC Megafunds For AI,AI Funding Frenzy,12,Mirelo raises $41M seed from Index and Andrees...,6.947508,False,[SiliconANGLE](https://siliconangle.com/2025/1...,4
8,VC Megafunds For AI,AI Funding Frenzy,35,General Intelligence Company raises $8.7M seed...,7.082178,False,[PYMNTS](https://www.pymnts.com/news/artificia...,4
9,AI Impact On Society,AI Overreach and Backlash,60,Amazon adds 'Ask This Book' Q&A to Kindle with...,6.065633,False,[Lifehacker](https://lifehacker.com/tech/amazo...,2


In [29]:
Markdown(state.display_newsletter().replace("$","\\\$"))


## AI: Deals, Doubts, Discipline
- Disney signs \\$1B, 3-year OpenAI deal with one-year Sora exclusivity for 200+ characters - [TechCrunch](https:\/\/techcrunch.com\/2025\/12\/15\/disneys-openai-deal-is-exclusive-for-just-one-year-then-its-open-season\/) [Inside The Magic](https:\/\/insidethemagic.net\/2025\/12\/creators-already-making-horrifying-videos-using-disney-ip-on-sora-and-openai-rl1\/)
- Notion files \\$300M tender offer, values itself at \\$11B after \\$600M ARR with half from AI products - [Techmeme](https://www.techmeme.com/251215/p35#a251215p35)
- Salesforce launches seat-based 'Agentic Enterprise' AI license, to absorb near-term losses to lock customers - [The Register](https://www.theregister.com/2025/12/15/salesforce_ai_monetization/)
- CrowdStrike launches Falcon AI Detection & Response to block prompt injection and log AI interactions - [SiliconANGLE](https://siliconangle.com/2025/12/15/falcon-ai-detection-response-expands-crowdstrike-platform-protect-ai-interactions/)
- Creative Commons tentatively backs pay-to-crawl, urges CDNs to adopt RSL and protect public-interest access - [TechCrunch](https://techcrunch.com/2025/12/15/creative-commons-announces-tentative-support-for-ai-pay-to-crawl-systems/)
- FSOC flags cyber risk as systemic, recommends third-party exams and an AI governance working group - [PYMNTS](https://www.pymnts.com/economy/2025/fsoc-trains-its-lens-on-cyber-attacks-and-ai/)
- Google Search adds '+' upload to feed images/docs into Gemini for on-demand analysis - [CNET](https://www.cnet.com/tech/services-and-software/google-search-new-plus-button-upload-images-and-docs-search-with-ai/)
- Gallup: 45% of U.S. adults use AI at work; 23% weekly, 10% daily; many employees don't know if their employer adopted AI - [ZDNet](https://www.zdnet.com/article/gallup-survey-ai-use-increase-communication-gap-workers-employers/) [Business Insider](https://www.businessinsider.com/most-common-ai-use-workplace-gallup-poll-2025-12)
- AI hasn't transformed factory floors: fragmented shop-floor systems hinder adoption even as AI in manufacturing projects to grow from \\$34B (2025) to \\$155B by 2030 - [Fast Company](https://www.fastcompany.com/91446538/ai-manufacturing-factories)
- Motif publishes open-weight Motif-2-12.7B-Reasoning; Korea benchmark ranks it top, outperforms GPT-5.1 on some tests, and shares four enterprise LLM training lessons - [VentureBeat](https:\/\/venturebeat.com\/ai\/korean-ai-startup-motif-reveals-4-big-lessons-for-training-enterprise-llms)
- CFO Alliance: 2026 is finance's 'most pivotal year' in a decade; warns geopolitical, talent, tech and governance risks and urges AI investments tied to measurable ROI - [Fortune](https://fortune.com/2025/12/15/2026-year-of-execution-cfo-strategy/)
- AI-driven zero-click search and trusted signals favor incumbents (OpenAI-linked platforms, Amazon, Walmart); SMBs lean on marketplaces, social and paid ads - [Practical Ecommerce](https://www.practicalecommerce.com/ai-favors-ecommerce-giants-for-now)
- Opinion: AI could hollow out Madison Avenue, threatening agency jobs and reshaping advertising economics - [The Wall Street Journal](https://www.wsj.com/opinion/ai-is-about-to-empty-madison-avenue-58ab2ea2)
- Observers warn AI investment mania mirrors late‑1990s dot‑com bubble - [The Wall Street Journal](https://www.wsj.com/finance/stocks/the-eerie-parallels-between-ai-mania-and-the-dot-com-bubble-f99be6fe)
- Observers warn AI investment mania mirrors late‑1990s dot‑com bubble - [The Wall Street Journal](https://www.wsj.com/finance/stocks/the-eerie-parallels-between-ai-mania-and-the-dot-com-bubble-f99be6fe)
- Analysts pivot between Nvidia GPUs and Broadcom custom accelerators as AI portfolio rotation intensifies - [Biztoc](https://biztoc.com/x/0a64c1f29465e977)
- CFA Institute CEO says AI will empower India's investment professionals by collapsing market silos; CFA program to add AI ethics and use cases - [Times of India](https://economictimes.indiatimes.com/news/company/corporate-trends/ai-will-empower-indias-investment-professionals-cfa-institute-ceo/articleshow/125965713.cms)
## AI Infrastructure Arms Race
- Nvidia acquires SchedMD, keeps Slurm open-source and launches Nemotron 3 Nano/Super/Ultra - [TechCrunch](https://techcrunch.com/2025/12/15/nvidia-bulks-up-open-source-offerings-with-an-acquisition-and-new-open-ai-models/) [Reuters](https://www.reuters.com/business/nvidia-buys-ai-software-provider-schedmd-expand-open-source-ai-push-2025-12-15/)
- Dell raises FY26 AI-systems guidance to \\$25B, citing token-driven demand and warning of memory supply risks - [SiliconANGLE](https://siliconangle.com/2025/12/15/ai-factory-growth-signal-dell-cant-ignore-aifactoriesdatacenters/)
- Q4 2025 HDD contract prices rise about 4% QoQ on hyperscaler and China PC demand - [Tom's Hardware](https://www.tomshardware.com/pc-components/hdds/hdd-prices-spike-as-ai-infrastructure-and-chinas-pc-push-collide-hard-drives-record-biggest-price-in-eight-quarters-suppliers-warn-pressure-will-continue)
- Tiiny AI Pocket Lab mini‑PC offers 12‑core ARM, ~190 TOPS AI module and 80GB RAM for on-device 10B–120B LLMs - [TechRadar](https://www.techradar.com/pro/worlds-smallest-ai-supercomputer-achieves-world-record-with-120b-parameter-llm-support-on-device-what-i-dont-understand-though-is-how-it-does-ota-hardware-upgrades)
- Broadcom shares drop about 10% after weaker AI product backlog and margin warning - [Yahoo](https://finance.yahoo.com/news/todays-market-p-500-falls-183508150.html)
- Chinese orders for Nvidia H200 GPUs outstrip supply as Nvidia considers adding capacity - [Investor's Business Daily](https://www.investors.com/news/technology/nvidia-stock-high-demand-china-h200-chips/)
## AI Overreach and Backlash
- GNOME bans suspected AI‑written Shell extensions, citing reviewer overload - [The Verge](https://www.theverge.com/news/844655/gnome-linux-ai-shell-extensions-ban)
- LG adds non‑removable Microsoft Copilot tile to webOS TVs - [Tom's Hardware](https://www.tomshardware.com/service-providers/tv-providers/lg-tv-update-adds-non-removable-microsoft-copilot-app-to-webos)
- Google declines a standardized news API, opting for traffic-based partnerships with 3,000+ publishers worldwide - [ZDNet](https://www.zdnet.com/article/google-ai-publisher-impact/)
- Google's 'AI Mode' synthesizes recipes without attribution, slashing food bloggers' traffic - [The Guardian](https://www.theguardian.com/technology/2025/dec/15/google-ai-recipes-food-bloggers)
- Amazon pulls AI‑generated video recaps after inaccurate 'Fallout' summary - [Futurism](https://futurism.com/artificial-intelligence/amazon-deletes-ai-recap-fallout)
- Elon Musk warns AI and robotics could make work, wages and money 'disappear' within 10-20 years - [Fortune](https://fortune.com/2025/12/15/billionaire-elon-musk-say-that-money-will-disappear-in-the-future-as-ai-makes-work-and-salaries-irrelevant-sorry-six-figure-earners/)
- Amazon adds 'Ask This Book' Q&A to Kindle without an author opt‑out - [Lifehacker](https://lifehacker.com/tech/amazon-kindle-ask-this-book)
- YouTube sees toddlers monetizing AI‑generated baby videos, sparking safety concerns - [Bloomberg](https://www.bloomberg.com/news/articles/2025-12-15/youtube-y-la-ia-auge-del-contenido-para-bebes-preocupa-a-expertos-y-padres)
- Joseph Gordon‑Levitt urges guardrails and pay after AI firms train on stolen creator work - [Fortune](https://fortune.com/2025/12/15/joseph-gordon-levitt-ai-laws-dystopian/)
- GitHub user zoicware releases tool to remove Windows 11 AI features (Copilot) - disables registry keys and blocks reinstall on 25H2+ - [PC Gamer](https://www.pcgamer.com/software/windows/if-you-want-to-scrub-windows-11s-ai-features-off-the-face-of-the-earth-some-legend-has-made-a-tool-that-does-just-that/)
- Cloudflare Radar: 2025 internet traffic up 19%; ChatGPT leads AI services; Starlink traffic up 2.3x - [Cloudflare Radar](https://radar.cloudflare.com/year-in-review/2025)
## AI Data-Center Debt Squeeze
- Big Tech shifts AI data-center debt onto smaller operators and lenders - [The New York Times](https://www.nytimes.com/2025/12/15/technology/ai-risks-debt.html)
- Tech CDS buying surges 90% since September as investors target Oracle, Meta, CoreWeave - [Gizmodo](https://gizmodo.com/investors-are-using-the-same-tool-as-the-big-short-guys-to-hedge-against-an-ai-bubble-2000699860)
- 86% of Nvidia's \\$3.84B AI portfolio tied to CoreWeave as CoreWeave shares slide 60% - [247wallst](https://247wallst.com/investing/2025/12/14/nvidias-biggest-ai-bet-is-crashing-its-secret-portfolio-can-it-recover/)
- Data-center energy vendor cites strong AI order momentum, raises 2028 guidance - [CNBC](https://www.cnbc.com/2025/12/15/were-booking-some-big-profits-in-an-ai-data-center-energy-play-after-the-stocks-big-rally.html)
- Energy Transfer advances Lake Charles LNG, AI-linked projects and Hugh Brinson Pipeline; forecasts \\$99.8B revenue and \\$6.7B earnings by 2028 - [Yahoo](https://finance.yahoo.com/news/did-energy-transfer-ai-focused-210845617.html)
- JPMorgan calls Oracle a 'show-me' stock after large AI debt bet - [Bloomberg](https://www.bloomberg.com/news/articles/2025-12-15/oracle-a-show-me-story-after-big-ai-debt-bet-jpmorgan-says)
## AI Funding Frenzy
- Lightspeed raises \\$9B, earmarks \\$3.3B to back Anthropic, xAI and Mistral - [The New York Times](https://www.nytimes.com/2025/12/15/technology/ai-venture-capital-big-funds.html)
- Chai raises \\$130M Series B to scale its AI drug‑discovery platform - [Bloomberg](https://www.bloomberg.com/news/videos/2025-12-15/ai-drug-discovery-firm-chai-raises-130-million-series-b-video)
- General Intelligence Company raises \\$8.7M seed to build 'Cofounder' agentic AI targeting autonomy by H1 2026 - [PYMNTS](https://www.pymnts.com/news/artificial-intelligence/2025/one-person-companies-generative-media-ai-funding-spans-stack/)
- Mirelo raises \\$41M seed from Index and Andreessen Horowitz, launches SFX 1.5 audio API - [SiliconANGLE](https://siliconangle.com/2025/12/15/audio-generation-startup-mirelo-nabs-41m-investment/)
## Washington's AI Playbook
- Trump orders federal block on state AI rules, urges attorneys general to sue - [NPR](https:\/\/www.npr.org\/2025\/12\/14\/nx-s1-5644086\/president-trump-moves-to-block-state-ai-laws-as-new-york-pushes-tougher-rules)
- White House signs order to ease AI innovation barriers, plans 1,000 hires; experts warn of risks - [Politico](https:\/\/www.politico.com\/news\/2025\/12\/15\/the-white-houses-unabashed-embrace-of-ai-00690885)
## AI Models Meet Markets
- GPT-4 passes the Uniform Bar Exam, but models still hallucinate citations and struggle on hardest problems - [Technology Review](https://www.technologyreview.com/2025/12/15/1129181/ai-might-not-be-coming-for-lawyers-jobs-anytime-soon/)
- Google's FACTS benchmark finds top chatbots only about 62–69% factual; Gemini 3 Pro tops at 69% - [Digital Trends](https://www.digitaltrends.com/computing/google-finds-ai-chatbots-are-only-69-accurate-at-best/)
- ChatGPT micro-cap trading experiment loses 32.9% over six months with 50.3% drawdown and annualized Sharpe -1.13 - [Hacker Noon](https:\/\/hackernoon.com\/can-chatgpt-outperform-the-market-week-19?source=rss)
- Google's Gemini Deep Research can generate integrated visual reports - custom images, charts and interactive simulations - available to AI Ultra subscribers - [Google Blog](https://blog.google/products/gemini/visual-reports/)
## Robotaxi Rush
- Tesla begins driverless robotaxi tests in Austin; FSD v14.2 adds Grok voice navigation - [Yahoo](https://finance.yahoo.com/news/tesla-stock-pops-as-robotaxi-testing-with-no-safety-driver-confirmed-164047565.html) [Next Big Future](https://www.nextbigfuture.com/2025/12/unaustin-tx.html)
- Rivian unveils RAP1 autonomy stack with 1,600-TOPS chip, offers autonomy for \\$2,500 or \\$49.99/mo - [Fortune](https://fortune.com/2025/12/15/rivian-ceo-ev-maker-new-chips-autonomous-driving-robotaxi/)
## AI Slop Goes Viral
- Merriam-Webster names 'slop' 2025 Word of the Year for low-quality AI content - [Ars Technica](https://arstechnica.com/ai/2025/12/merriam-webster-crowns-slop-word-of-the-year-as-ai-content-floods-internet/)
- Oxford, Macquarie and Cambridge flag AI-era terms such as 'rage bait' and 'AI slop' - [The Independent](https://www.independent.co.uk/tech/word-year-online-rage-bait-67-ai-slop-b2883457.html)
## AI Harms and Data Leaks
- Logs show OpenAI's OAI-SearchBot used Certificate Transparency logs to find sites and fetch robots.txt - [Ben Jojo](https://benjojo.co.uk/u/benjojo/h/Gxy2qrCkn1Y327Y6D3)
- Reports: third parties are harvesting and selling private ChatGPT and Gemini conversations - [Forbes](https://www.forbes.com/sites/zakdoffman/2025/12/15/how-your-private-chatgpt-and-gemini-chats-are-sold-for-profit/)
## AI's Ripples: Tech, Jobs, Markets
- Klarna opens Agentic Product Protocol API: feed of 100M products, 400M prices across 12 markets for AI agents - [PYMNTS](https://www.pymnts.com/news/artificial-intelligence/2025/klarna-launches-open-standard-making-products-discoverable-ai-agents/)
- Bodycam shows AI facial-recognition misidentified trucker, leading to 11-hour detention - [Dexerto](https://www.dexerto.com/entertainment/bodycam-footage-reveals-moment-innocent-man-is-arrested-after-ai-misidentifies-him-3294374/)


In [30]:
# User prompt to run workflow
# user_prompt = "Run step 9, finalize newsletter"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("finalize_newsletter")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


18:28:25 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Step 9: write title
18:28:25 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Compiled 58 items into markdown input
18:28:25 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Successfully retrieved prompt 'newsagent/generate_newsletter_title' from Langfuse
18:28:25 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Parsed prompt 'newsagent/generate_newsletter_title': model=gpt-5-mini, reasoning_effort=high, system_len=1082, user_len=225


▶ Starting Step 9: finalize_newsletter


18:29:04 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Generated title: Disney-OpenAI $1B Deal; Infrastructure Arms Race Strains Markets; Regulators, Creators Push Back
18:29:04 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Step 9b: Starting critic-optimizer loop
18:29:04 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Successfully retrieved prompt 'newsagent/critique_newsletter' from Langfuse
18:29:04 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Parsed prompt 'newsagent/critique_newsletter': model=gpt-5-mini, reasoning_effort=medium, system_len=2758, user_len=647
18:29:04 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Successfully retrieved prompt 'newsagent/improve_newsletter' from Langfuse
18:29:04 | NewsletterAgent.test_newsletter_20251215175959759192 | INFO | Parsed prompt 'newsagent/improve_newsletter': model=gpt-5-mini, reasoning_effort=high, system_len=1320, user_len=195
18:29:04 | Newslette

⏱️  Total execution time: 539.60s
📊 Final result:
🎉 Step 9 finalize_newsletter completed successfully!
⭐ Final quality score: 7.6/10
🔄 Completed 2 critic-optimizer iteration(s)
📰 Newsletter stored in persistent state and emailed
✅ Workflow complete! All 9 steps finished successfully.


In [31]:
display(Markdown(state.final_newsletter.replace("$", "\\\$")))

# Disney-OpenAI \\$1B Deal, Infrastructure Strain, and Creator Pushback

## Big Bets & Partnerships

- Disney signs \\$1B, 3-year OpenAI deal; Sora exclusivity for 1 year on 200+ characters — [TechCrunch](https:\/\/techcrunch.com\/2025\/12\/15\/disneys-openai-deal-is-exclusive-for-just-one-year-then-its-open-season\/) [Inside The Magic](https:\/\/insidethemagic.net\/2025\/12\/creators-already-making-horrifying-videos-using-disney-ip-on-sora-and-openai-rl1\/)
- Nvidia acquires SchedMD, keeps Slurm open-source; launches Nemotron 3 Nano/Super/Ultra — [TechCrunch](https://techcrunch.com/2025/12/15/nvidia-bulks-up-open-source-offerings-with-an-acquisition-and-new-open-ai-models/) [Reuters](https://www.reuters.com/business/nvidia-buys-ai-software-provider-schedmd-expand-open-source-ai-push-2025-12-15/)

## Compute Arms Race

- Dell raises FY26 AI-systems guidance to \\$25B; cites token-driven demand, warns of memory supply risks — [SiliconANGLE](https://siliconangle.com/2025/12/15/ai-factory-growth-signal-dell-cant-ignore-aifactoriesdatacenters/)
- Q4 2025 HDD contract prices rise ~4% QoQ on hyperscaler and China PC demand — [Tom's Hardware](https://www.tomshardware.com/pc-components/hdds/hdd-prices-spike-as-ai-infrastructure-and-chinas-pc-push-collide-hard-drives-record-biggest-price-in-eight-quarters-suppliers-warn-pressure-will-continue)
- Tiiny AI Pocket Lab mini‑PC supports on-device 10B–120B LLMs with 12-core ARM, ~190 TOPS AI module and 80GB RAM — [TechRadar](https://www.techradar.com/pro/worlds-smallest-ai-supercomputer-achieves-world-record-with-120b-parameter-llm-support-on-device-what-i-dont-understand-though-is-how-it-does-ota-hardware-upgrades)
- Broadcom shares fall about 10% after weaker AI product backlog and margin warning — [Yahoo](https://finance.yahoo.com/news/todays-market-p-500-falls-183508150.html)
- Chinese orders for Nvidia H200 GPUs outstrip supply as Nvidia considers adding capacity — [Investor's Business Daily](https://www.investors.com/news/technology/nvidia-stock-high-demand-china-h200-chips/)
- Cloudflare Radar: 2025 internet traffic up 19%; ChatGPT leads AI services; Starlink traffic up 2.3x — [Cloudflare Radar](https://radar.cloudflare.com/year-in-review/2025)

## Products & Protections

- Salesforce introduces seat-based 'Agentic Enterprise' AI license; plans to absorb near-term losses — [The Register](https://www.theregister.com/2025/12/15/salesforce_ai_monetization/)
- CrowdStrike launches Falcon AI Detection & Response to block prompt injection and log AI interactions — [SiliconANGLE](https://siliconangle.com/2025/12/15/falcon-ai-detection-response-expands-crowdstrike-platform-protect-ai-interactions/)
- Google Search adds '+' upload to feed images and docs into Gemini for on-demand analysis — [CNET](https://www.cnet.com/tech/services-and-software/google-search-new-plus-button-upload-images-and-docs-search-with-ai/)
- Amazon adds 'Ask This Book' Q&A to Kindle without an author opt‑out — [Lifehacker](https://lifehacker.com/tech/amazon-kindle-ask-this-book)

## Policy Playbook

- Creative Commons tentatively backs pay-to-crawl; urges CDNs to adopt RSL and protect public-interest access — [TechCrunch](https://techcrunch.com/2025/12/15/creative-commons-announces-tentative-support-for-ai-pay-to-crawl-systems/)
- FSOC flags cyber risk as systemic; recommends third-party exams and an AI governance working group — [PYMNTS](https://www.pymnts.com/economy/2025/fsoc-trains-its-lens-on-cyber-attacks-and-ai/)
- Trump orders federal block on state AI rules, urges attorneys general to sue — [NPR](https:\/\/www.npr.org\/2025\/12\/14\/nx-s1-5644086\/president-trump-moves-to-block-state-ai-laws-as-new-york-pushes-tougher-rules)
- White House signs order to ease AI innovation barriers, plans 1,000 hires; experts warn of risks — [Politico](https:\/\/www.politico.com\/news\/2025\/12\/15\/the-white-houses-unabashed-embrace-of-ai-00690885)

## Markets & Money

- Notion files \\$300M tender offer; values itself at \\$11B after \\$600M ARR, half from AI products — [Techmeme](https://www.techmeme.com/251215/p35#a251215p35)
- Gallup: 45% of U.S. adults use AI at work; 23% weekly, 10% daily; many employees unsure if their employer adopted AI — [ZDNet](https://www.zdnet.com/article/gallup-survey-ai-use-increase-communication-gap-workers-employers/) [Business Insider](https://www.businessinsider.com/most-common-ai-use-workplace-gallup-poll-2025-12)
- AI hasn't transformed factory floors; fragmented shop-floor systems hinder adoption even as manufacturing AI projects grow from \\$34B (2025) to \\$155B by 2030 — [Fast Company](https://www.fastcompany.com/91446538/ai-manufacturing-factories)
- Analysts pivot between Nvidia GPUs and Broadcom custom accelerators as AI portfolio rotation intensifies — [Biztoc](https://biztoc.com/x/0a64c1f29465e977)
- CFO Alliance: 2026 is finance's 'most pivotal year' in a decade; warns geopolitical, talent, tech and governance risks and urges AI investments tied to measurable ROI — [Fortune](https://fortune.com/2025/12/15/2026-year-of-execution-cfo-strategy/)
- CFA Institute CEO says AI will empower India's investment professionals; CFA program to add AI ethics and use cases — [Times of India](https://economictimes.indiatimes.com/news/company/corporate-trends/ai-will-empower-indias-investment-professionals-cfa-institute-ceo/articleshow/125965713.cms)

## Data-Center Debt Squeeze

- Big Tech shifts AI data-center debt onto smaller operators and lenders — [The New York Times](https://www.nytimes.com/2025/12/15/technology/ai-risks-debt.html)
- Tech CDS buying surges 90% since September as investors target Oracle, Meta, CoreWeave — [Gizmodo](https://gizmodo.com/investors-are-using-the-same-tool-as-the-big-short-guys-to-hedge-against-an-ai-bubble-2000699860)
- 86% of Nvidia's \\$3.84B AI portfolio tied to CoreWeave as CoreWeave shares slide 60% — [247wallst](https://247wallst.com/investing/2025/12/14/nvidias-biggest-ai-bet-is-crashing-its-secret-portfolio-can-it-recover/)
- Data-center energy vendor cites strong AI order momentum, raises 2028 guidance — [CNBC](https://www.cnbc.com/2025/12/15/were-booking-some-big-profits-in-an-ai-data-center-energy-play-after-the-stocks-big-rally.html)
- Energy Transfer advances Lake Charles LNG, AI-linked projects and Hugh Brinson Pipeline; forecasts \\$99.8B revenue and \\$6.7B earnings by 2028 — [Yahoo](https://finance.yahoo.com/news/did-energy-transfer-ai-focused-210845617.html)
- JPMorgan calls Oracle a 'show-me' stock after large AI debt bet — [Bloomberg](https://www.bloomberg.com/news/articles/2025-12-15/oracle-a-show-me-story-after-big-ai-debt-bet-jpmorgan-says)

## Funding Frenzy

- Lightspeed raises \\$9B, earmarks \\$3.3B to back Anthropic, xAI and Mistral — [The New York Times](https://www.nytimes.com/2025/12/15/technology/ai-venture-capital-big-funds.html)
- Chai raises \\$130M Series B to scale its AI drug‑discovery platform — [Bloomberg](https://www.bloomberg.com/news/videos/2025-12-15/ai-drug-discovery-firm-chai-raises-130-million-series-b-video)
- General Intelligence Company raises \\$8.7M seed to build 'Cofounder' agentic AI targeting autonomy by H1 2026 — [PYMNTS](https://www.pymnts.com/news/artificial-intelligence/2025/one-person-companies-generative-media-ai-funding-spans-stack/)
- Mirelo raises \\$41M seed from Index and Andreessen Horowitz; launches SFX 1.5 audio API — [SiliconANGLE](https://siliconangle.com/2025/12/15/audio-generation-startup-mirelo-nabs-41m-investment/)

## Models & Benchmarks

- GPT-4 passes the Uniform Bar Exam but models still hallucinate citations and struggle on hardest problems — [Technology Review](https://www.technologyreview.com/2025/12/15/1129181/ai-might-not-be-coming-for-lawyers-jobs-anytime-soon/)
- Google's FACTS benchmark finds top chatbots only about 62–69% factual; Gemini 3 Pro tops at 69% — [Digital Trends](https://www.digitaltrends.com/computing/google-finds-ai-chatbots-are-only-69-accurate-at-best/)
- ChatGPT micro-cap trading experiment loses 32.9% over six months with 50.3% drawdown and annualized Sharpe -1.13 — [Hacker Noon](https:\/\/hackernoon.com\/can-chatgpt-outperform-the-market-week-19?source=rss)
- Google's Gemini Deep Research can generate integrated visual reports — custom images, charts and interactive simulations — available to AI Ultra subscribers — [Google Blog](https://blog.google/products/gemini/visual-reports/)

## Robotaxi Rush

- Tesla begins driverless robotaxi tests in Austin; FSD v14.2 adds Grok voice navigation — [Yahoo](https://finance.yahoo.com/news/tesla-stock-pops-as-robotaxi-testing-with-no-safety-driver-confirmed-164047565.html) [Next Big Future](https://www.nextbigfuture.com/2025/12/unaustin-tx.html)
- Rivian unveils RAP1 autonomy stack with 1,600-TOPS chip; offers autonomy for \\$2,500 or \\$49.99/mo — [Fortune](https://fortune.com/2025/12/15/rivian-ceo-ev-maker-new-chips-autonomous-driving-robotaxi/)

## Platform Overreach

- Google declines a standardized news API, opting for traffic-based partnerships with 3,000+ publishers worldwide — [ZDNet](https://www.zdnet.com/article/google-ai-publisher-impact/)
- LG adds a non-removable Microsoft Copilot tile to webOS TVs — [Tom's Hardware](https://www.tomshardware.com/service-providers/tv-providers/lg-tv-update-adds-non-removable-microsoft-copilot-app-to-webos)
- Google's 'AI Mode' synthesizes recipes without attribution, slashing food bloggers' traffic — [The Guardian](https://www.theguardian.com/technology/2025/dec/15/google-ai-recipes-food-bloggers)
- GNOME bans suspected AI‑written Shell extensions, citing reviewer overload — [The Verge](https://www.theverge.com/news/844655/gnome-linux-ai-shell-extensions-ban)

## Creators Push Back

- Joseph Gordon‑Levitt urges guardrails and pay after AI firms train on stolen creator work — [Fortune](https://fortune.com/2025/12/15/joseph-gordon-levitt-ai-laws-dystopian/)
- YouTube sees toddlers monetizing AI‑generated baby videos, sparking safety concerns — [Bloomberg](https://www.bloomberg.com/news/articles/2025-12-15/youtube-y-la-ia-auge-del-contenido-para-bebes-preocupa-a-expertos-y-padres)
- Amazon pulls AI‑generated video recaps after an inaccurate 'Fallout' summary — [Futurism](https://futurism.com/artificial-intelligence/amazon-deletes-ai-recap-fallout)
- GitHub user zoicware releases a tool to remove Windows 11 AI features (Copilot); disables registry keys and blocks reinstall on 25H2+ — [PC Gamer](https://www.pcgamer.com/software/windows/if-you-want-to-scrub-windows-11s-ai-features-off-the-face-of-the-earth-some-legend-has-made-a-tool-that-does-just-that/)

## Commerce & Agents

- AI-driven zero-click search and trusted signals favor incumbents (OpenAI-linked platforms, Amazon, Walmart); SMBs lean on marketplaces, social and paid ads — [Practical Ecommerce](https://www.practicalecommerce.com/ai-favors-ecommerce-giants-for-now)
- Klarna opens Agentic Product Protocol API: feed of 100M products, 400M prices across 12 markets for AI agents — [PYMNTS](https://www.pymnts.com/news/artificial-intelligence/2025/klarna-launches-open-standard-making-products-discoverable-ai-agents/)

## Harms & Data Leaks

- Logs show OpenAI's OAI-SearchBot used Certificate Transparency logs to find sites and fetch robots.txt — [Ben Jojo](https://benjojo.co.uk/u/benjojo/h/Gxy2qrCkn1Y327Y6D3)
- Reports: third parties are harvesting and selling private ChatGPT and Gemini conversations — [Forbes](https://www.forbes.com/sites/zakdoffman/2025/12/15/how-your-private-chatgpt-and-gemini-chats-are-sold-for-profit/)
- Bodycam footage shows AI facial-recognition misidentified a trucker, leading to an 11-hour detention — [Dexerto](https://www.dexerto.com/entertainment/bodycam-footage-reveals-moment-innocent-man-is-arrested-after-ai-misidentifies-him-3294374/)

## Slop Goes Viral

- Merriam-Webster names 'slop' 2025 Word of the Year for low-quality AI content — [Ars Technica](https://arstechnica.com/ai/2025/12/merriam-webster-crowns-slop-word-of-the-year-as-ai-content-floods-internet/)
- Oxford, Macquarie and Cambridge flag AI-era terms such as 'rage bait' and 'AI slop' — [The Independent](https://www.independent.co.uk/tech/word-year-online-rage-bait-67-ai-slop-b2883457.html)

## Opinion & Analysis

- Opinion: AI could hollow out Madison Avenue, threatening agency jobs and reshaping advertising economics — [The Wall Street Journal](https://www.wsj.com/opinion/ai-is-about-to-empty-madison-avenue-58ab2ea2)
- Observers warn AI investment mania mirrors late‑1990s dot‑com bubble — [The Wall Street Journal](https://www.wsj.com/finance/stocks/the-eerie-parallels-between-ai-mania-and-the-dot-com-bubble-f99be6fe)
- Elon Musk warns AI and robotics could make work, wages and money 'disappear' within 10–20 years — [Fortune](https://fortune.com/2025/12/15/billionaire-elon-musk-say-that-money-will-disappear-in-the-future-as-ai-makes-work-and-salaries-irrelevant-sorry-six-figure-earners/)


In [32]:
print(state.get_workflow_status_report())



WORKFLOW STATUS
Progress: 100.0% (9/9 complete)
Status Summary: 9 complete, 0 started, 0 failed, 0 not started
Status: All steps complete or started

Step Details:
  Step 0: Gather URLs: complete
    Status: ✅ Step 1 gather_urls completed successfully! Gathered 838 articles from 17 sources.

📊 Articles stored in persistent state: 707
  Step 1: Filter URLs: complete
    Status: ✅ Step 2 filter_urls completed successfully! Removed 484 duplicate URLs, classified 223 new articles, found 103 AI-related.
  Step 2: Download Articles: complete
    Status: ✅ Step 3 download_articles completed successfully! Downloaded 86 AI-related articles with 86% success rate.
📊 Average article length: 4653 characters
🔗 Content stored in persistent state.
  Step 3: Extract Summaries: complete
    Status: ✅ Step 4 extract_summaries completed successfully! Generated AI-powered summaries for 100/100 articles.
💾 Summaries stored in headline DataFrame.
  Step 4: Rate Articles: complete
    Status: ✅ Step 5 rate_ar

In [33]:
state.print_workflow_status()



📊 WORKFLOW STATUS
📈 Progress: 100.0% (9/9 complete)
🎉 All steps complete!

📋 Step Details:
  ✅ Step 0: Gather URLs            | complete
      📝 Status: ✅ Step 1 gather_urls completed successfully! Gathered 838 articles from 17 sources.

📊 Articles stored in persistent state: 707
  ✅ Step 1: Filter URLs            | complete
      📝 Status: ✅ Step 2 filter_urls completed successfully! Removed 484 duplicate URLs, classified 223 new articles, found 103 AI-related.
  ✅ Step 2: Download Articles      | complete
      📝 Status: ✅ Step 3 download_articles completed successfully! Downloaded 86 AI-related articles with 86% success rate.
📊 Average article length: 4653 characters
🔗 Content stored in persistent state.
  ✅ Step 3: Extract Summaries      | complete
      📝 Status: ✅ Step 4 extract_summaries completed successfully! Generated AI-powered summaries for 100/100 articles.
💾 Summaries stored in headline DataFrame.
  ✅ Step 4: Rate Articles          | complete
      📝 Status: ✅ Step 5 rat